In [ ]:
import io, os, sys, types
from IPython import get_ipython
from Utils import NotebookFinder
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell
import numpy as np
import matplotlib.pyplot as plt
import csv
from env import Env
from RL_brain import DeepQNetwork
from  Configuration import null,setDbgPrint
from runs import sim
import tensorflow as tf

In [ ]:
sys.meta_path.append(NotebookFinder())

In [ ]:
def trainrun(Kcount,cfg=None):
    # For ease of access to run parameters.
    step = 0
    numEpisodes = cfg['numEpisodesTrain']
    #maxSteps = cfg['maxStepsTrain']
    startLearnStep = cfg['startLearnStep']
    learnPeriod = cfg.get('learnPeriod', 5)
    dbgPrint = cfg.get('dbgPrint', null)
    statusPeriod = cfg.get('statusPeriod', 1)
    statusPeriod = sim['cfg']['run']['statusPeriod']
    dbgPrint = setDbgPrint(sim['cfg']['run'].get('dbgPrint'))
    test = sim['cfg']['run']['test']
    # Init. environment.
    env = Env(log=dbgPrint, statusPeriod=statusPeriod,Kcount=Kcount)

    # For ease of access to DQN config. parameters.
    learningRate = sim['cfg']['dqn']['learningRate']
    rewardDecay = sim['cfg']['dqn']['rewardDecay']
    eGreedy = sim['cfg']['dqn']['eGreedy']
    e_greedy_increment = sim['cfg']['dqn']['eGreedyincrement']
    replaceTargetIter = sim['cfg']['dqn']['replaceTargetIter']
    rewardDecay = sim['cfg']['dqn']['learningRate']
    memorySize = sim['cfg']['dqn']['memorySize']
    if Kcount == 0:
        maxSteps =  1887155
    elif Kcount == 1:
        maxSteps = 1887159
    else:
        maxSteps = 1887164
    RL = DeepQNetwork(n_actions=env.n_actions,
                      n_features=env.n_features,
                      learning_rate=learningRate,
                      reward_decay=rewardDecay,
                      e_greedy=eGreedy,
                      replace_target_iter=replaceTargetIter,
                      e_greedy_increment=e_greedy_increment,
                      memory_size=memorySize,
                      dbgPrint=dbgPrint,
                      statusPeriod=statusPeriod
                      )
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    F1_history =[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Pr_history=[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Rc_history = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    count = np.zeros((15, 4))
    for episode in range(1,numEpisodes+1):
        step = 0
        env_state_1 = env.reset()
        env.count_history = []
        countk = 0
        countre = 0
        while step < maxSteps:
            #print("step = {}".format(step))
            # fresh env
            # Output current observation:
            #print('{}: current env = {}\n'.format(env.time, env.__dict__))

            env_state = env_state_1["feature"]
            # RL choose action based on env_state
            env_state = np.hstack(env_state)
            action = RL.choose_action(env_state)
         #   dbgPrint('{}: action_ = {}, observation = {}\n'.format(env.time, action_, observation), period=statusPeriod, counter=env.time)
            # print(observation)
            # RL take action and get next observation and reward
            observation_, reward,label = env.step(action,env_state_1)
            label=int(label)
            action=int(action)
            #print("当前的特征为{}，标签为{}，选择为{}，奖励为{}".format(env_state,label,action,reward))
            #
            counti =0
            for i in [label,action]:

                if label == i and action == i:  # TP
                    #print('kkkkkk TP=', count[i][0])
                    count[i][0] += 1
                    if counti == 1:
                        count[i][0] -=1
                    counti += 1
                elif action == i and label != i:  # FP
                    #print('kkkkkk FP', count[i][1])
                    count[i][1] += 1
                elif action != i and label != i:  # TN
                    #print('kkkkkk TN', count[i][2])
                    count[i][2] += 1
                elif action != i and label == i:  # FN
                   # print('kkkkkk FN', count[i][3])
                    count[i][3] += 1
                if ((count[i][0] + count[i][3]) != 0 and (count[i][0] + count[i][1]) != 0):
                    Pr = count[i][0] / (count[i][0] + count[i][1])
                    Rc = count[i][0] / (count[i][0] + count[i][3])
                    F1 = 2 / (1 / Pr + 1 / Rc)
                    Rc_history[i].append(count[i][0] / (count[i][0] + count[i][3]))
                    Pr_history[i].append(count[i][0] / (count[i][0] + count[i][1]))
                    F1_history[i].append(F1)
                    #print('{}___Rc = {},Pr = {},F1 = {}'.format(i, Rc, Pr, F1))
                   # print("RC_history[2]={},Pr_history={}".format(Rc_history[2],Pr_history[2]))
            # if label != 0 and action != 0:
            #      TPcount += 1
            # elif action != 0 and label == 0:
            #      FPcount += 1
            # elif action == 0 and label == 0:
            #      TNcount += 1
            # elif action == 0 and label != 0:
            #      FNcount += 1
            # if ((TPcount + FNcount) != 0 and (TPcount + FPcount) != 0):
            #     Pr = TPcount / (TPcount + FPcount)
            #     Rc = TPcount / (TPcount + FNcount)
            #     F1 = 2/(1/Pr+1/Rc)
            #     Pr_history.append(TPcount / (TPcount + FPcount))#It is the ratio of correctly classified attacks flows (TP), in front of all the classified flows (TP+FP).
            #     Rc_history.append(TPcount / (TPcount + FNcount))#It is the ratio of correctly classified attack flows (TP), in front of all generated flows (TP+FN).
            #     F1_history.append(2/(1/Pr+1/Rc))  #It is a harmonic combination of the precision and recall into a single measure.
            #     dbgPrint('Pr = {},Rc = {},F1 = {}'.format(Pr,Rc,F1),period = statusPeriod,counter=env.time)
            RL.store_transition(env_state, action, reward, observation_["feature"])
            # dbgPrint("env_state = {},label={}, action = {},reward = {}\n".format(env_state,label,action,reward), period=statusPeriod,
            #          counter=env.time)
           # print("env_state = {},label={}, action = {},reward = {},observation_ ={}\n".format(env_state,label,action,reward,observation_))
            if (step > startLearnStep) and (step % learnPeriod == 0):
                RL.learn()

            # swap observation
            #observation = observation_
            env_state_1 = observation_

            # break while loop when end of this episode
            step += 1

    plt.figure()
    for i in range(1,16):
        plt.subplot(1,3,1)
        plt.plot(
           Pr_history[i-1], '.-')
        plt.xlabel('Training Step')
        plt.ylabel('Pr_{}'.format(i-1))

        plt.subplot(1, 3, 2)
        plt.plot(Rc_history[i-1], '.-')
        plt.xlabel('Training Step')
        plt.ylabel('Rc_{}'.format(i-1))

        plt.subplot(1, 3, 3)
        plt.plot(
            F1_history[i-1], '.-')
        plt.xlabel('Training Step')
        plt.ylabel('F1')
        plt.grid(True)
        plt.show()
    plt.figure()
    plt.plot(
        np.divide( np.cumsum([1 if i >= 0 else 0 for i in env.count_history]), np.arange(len(env.count_history)) + 1),
        '.-')
    plt.xlabel('Training Step')
    plt.ylabel('Correct rate')
    plt.grid(True)
    plt.show()